# Semi-Supervised Rice Panicle Detection with YOLOv8

**In this notebook, we’ll implement a STAC (Self-Training with Augmented Consistency)-style self-training loop to reduce manual labeling effort and improve detection performance:**

1. **COCO→YOLO Conversion**  
   - Parse COCO annotations and convert bounding boxes to YOLOv8’s `<class> <x_center> <y_center> <width> <height>` format  
   - Organize images and labels into `train`, `valid`, and `test` splits

2. **Initial Teacher Model Training**  
   - Fine-tune a pretrained `yolov8s.pt` on the labeled data for 20 epochs  
   - Save the best weights as the “teacher” for pseudo-label generation

3. **Self-Training Loop (×5 Iterations)**  
   - **Pseudo-Labeling**: Use the current teacher to predict on the unlabeled train set (confidence ≥ 0.7) and save detections as pseudo-labels  
   - **Label Merging**: Combine ground-truth labels with high-confidence pseudo-labels in the student’s training folder  
   - **Student Training**: Train a student model (initialized from the teacher weights) for 10 epochs  
   - **Promotion**: Promote the student’s best weights to become the new teacher

4. **Final Evaluation**  
   - Run inference on the validation split with the final teacher model  
   - Report precision, recall, mAP, and other key metrics (no plots, just tabular output)

---

> **Why Semi-Supervised?**  
> By iteratively augmenting our labeled set with high-confidence pseudo-labels, we leverage both human annotations and model predictions—maximizing data utilization while minimizing manual effort.  


In [1]:
!pip -q install --upgrade --no-warn-script-location ultralytics albumentations pycocotools torchmetrics tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.4/369.4 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.3/477.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
!git clone -q https://github.com/ultralytics/yolov5.git

In [3]:
# ================================================================
# 0. Install dependencies & clone YOLOv5 for loss utils (optional)
# ================================================================
#!pip -q install --upgrade ultralytics==8.1.8 pycocotools albumentations torchmetrics tqdm
#!git clone -q https://github.com/ultralytics/yolov5.git

# ================================================================
# 1. Imports & Paths
# ================================================================
import json, shutil, random, warnings
from pathlib import Path
from ultralytics import YOLO
import yaml

# optionally import ComputeLoss if you need it later
import sys
sys.path.insert(0, "yolov5")
from utils.loss import ComputeLoss

# COCO‐format rice panicle dataset
BASE       = Path("/kaggle/input/annotated-rice-panicle-image-from-bangladesh")
TRAIN_IMG  = BASE/"train"
TRAIN_JSON = BASE/"train/_annotations.coco.json"
VALID_IMG  = BASE/"valid"
VALID_JSON = BASE/"valid/_annotations.coco.json"

# Working directories
WORK       = Path("/kaggle/working/rice_teacher_student")
SPLIT      = WORK/"0_yolo_split"
PSEUDO     = WORK/"1_pseudo"
STU_ROOT   = WORK/"2_student"

# create split dirs
for split in ["train","valid","test"]:
    (SPLIT/split/"images").mkdir(parents=True, exist_ok=True)
    (SPLIT/split/"labels").mkdir(parents=True, exist_ok=True)

# create pseudo and student dirs
(PSEUDO/"raw"/"labels").mkdir(parents=True, exist_ok=True)
for sub in ["train","valid"]:
    (STU_ROOT/sub/"images").mkdir(parents=True, exist_ok=True)
    (STU_ROOT/sub/"labels").mkdir(parents=True, exist_ok=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# ================================================================
# 2. Convert COCO → YOLOv8 format
# ================================================================
# read class names
cats = json.load(open(TRAIN_JSON))["categories"]
names = [c["name"] for c in sorted(cats, key=lambda x: x["id"])]
nc = len(names)

def coco2yolo(b, w, h):
    x,y,ww,hh = b
    return (x+ww/2)/w, (y+hh/2)/h, ww/w, hh/h

def convert_split(split, img_dir, ann_json):
    coco = json.load(open(ann_json))
    id2img = {img["id"]:img for img in coco["images"]}
    out_im = SPLIT/split/"images"
    out_lb = SPLIT/split/"labels"
    # write labels
    for ann in coco["annotations"]:
        img = id2img[ann["image_id"]]
        w,h = img["width"], img["height"]
        yb = coco2yolo(ann["bbox"], w, h)
        cls = ann["category_id"] - 1
        p = out_lb/f"{Path(img['file_name']).stem}.txt"
        with open(p, "a") as f:
            f.write(f"{cls} " + " ".join(f"{v:.6f}" for v in yb) + "\n")
    # copy images
    for img in coco["images"]:
        shutil.copy(Path(img_dir)/img["file_name"], out_im/img["file_name"])

# convert train, valid, test (using train as test if no separate split)
convert_split("train",  TRAIN_IMG,  TRAIN_JSON)
convert_split("valid",  VALID_IMG,  VALID_JSON)
convert_split("test",   TRAIN_IMG,  TRAIN_JSON)


In [5]:
# ================================================================
# 3. Write data YAMLs
# ================================================================
data_yaml = {
    'path': str(SPLIT),
    'train': 'train/images',
    'val':   'valid/images',
    'test':  'test/images',
    'nc':    nc,
    'names': names
}
(data_yaml_path := WORK/"data_rice.yaml").write_text(yaml.dump(data_yaml))

stu_yaml = {
    'path': str(STU_ROOT),
    'train': 'train/images',
    'val':   'valid/images',
    'nc':    nc,
    'names': names
}
(WORK/"data_rice_student.yaml").write_text(yaml.dump(stu_yaml))

134

In [6]:
# ================================================================
# 4. Initial teacher training
# ================================================================
teacher = YOLO("yolov8s.pt")
_ = teacher.train(
    data    = str(data_yaml_path),
    epochs  = 10,
    imgsz   = 640,
    batch   = 8,
    project = str(WORK),
    name    = "teacher",
    device  = 0
)
teacher_weights = WORK/"teacher"/"weights"/"best.pt"

100%|██████████| 21.5M/21.5M [00:00<00:00, 175MB/s]


Ultralytics 8.3.162 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/rice_teacher_student/data_rice.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=teacher, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

100%|██████████| 755k/755k [00:00<00:00, 22.3MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 125MB/s]


...

      10/10      2.61G      1.309       1.06        1.1         29        640: 100%|██████████| 220/220 [00:38<00:00,  5.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:01<00:00,  7.89it/s]

                   all        219       3962      0.655      0.721      0.737      0.478



10 epochs completed in 0.114 hours.
Optimizer stripped from /kaggle/working/rice_teacher_student/teacher/weights/last.pt, 22.5MB
Optimizer stripped from /kaggle/working/rice_teacher_student/teacher/weights/best.pt, 22.5MB

Validating /kaggle/working/rice_teacher_student/teacher/weights/best.pt...
Ultralytics 8.3.162 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:04<00:00,  2.84it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        219       3962      0.657      0.719      0.737      0.478
          Rice-Panicle        219       3962      0.657      0.719      0.737      0.478
Speed: 0.3ms preprocess, 5.8ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to /kaggle/working/rice_teacher_student/teacher


In [8]:
# ================================================================
# 5. Populate student valid split (required by YOLOv8.train)
# ================================================================
for img in (SPLIT/"valid"/"images").iterdir():
    shutil.copy(img, STU_ROOT/"valid"/"images"/img.name)
    lbl = SPLIT/"valid"/"labels"/f"{img.stem}.txt"
    shutil.copy(lbl, STU_ROOT/"valid"/"labels"/f"{img.stem}.txt")

In [9]:
# ================================================================
# 6. STAC‐style self‐training loop
# ================================================================
IMGZ        = 640
TAU         = 0.7
STAC_ITERS  = 5
STU_EPOCHS  = 10
BATCH       = 8

for it in range(STAC_ITERS):
    print(f"\n=== STAC iteration {it+1}/{STAC_ITERS} ===")

    # 6a) generate pseudo‐labels on train/images
    _ = YOLO(str(teacher_weights)).predict(
        source    = str(SPLIT/"train"/"images"),
        save_txt  = True,
        save_conf = True,
        conf      = TAU,
        project   = str(PSEUDO),
        name      = f"it{it}",
        device    = 0
    )
    RAW_LB = PSEUDO/f"it{it}"/"labels"

    # 6b) merge GT + pseudo into 2_student/train
    # clear old labels
    for f in (STU_ROOT/"train"/"labels").glob("*.txt"):
        f.unlink()
    # copy images
    for im in (SPLIT/"train"/"images").glob("*.jpg"):
        shutil.copy(im, STU_ROOT/"train"/"images"/im.name)
    # copy GT + append pseudo
    for gt in (SPLIT/"train"/"labels").glob("*.txt"):
        dst = STU_ROOT/"train"/"labels"/gt.name
        shutil.copy(gt, dst)
        pseudo_f = RAW_LB/gt.name
        if pseudo_f.exists():
            with open(dst, "a") as d, open(pseudo_f) as p:
                d.write(p.read())

    # 6c) train student from last teacher_weights
    student = YOLO(str(teacher_weights))
    _ = student.train(
        data    = str(WORK/"data_rice_student.yaml"),
        epochs  = STU_EPOCHS,
        imgsz   = IMGZ,
        batch   = BATCH,
        project = str(WORK),
        name    = f"student_it{it}",
        device  = 0
    )

    # promote student → teacher
    teacher_weights = WORK/f"student_it{it}"/"weights"/"best.pt"





=== STAC iteration 1/5 ===

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1754 /kaggle/working/rice_teacher_student/0_yolo_split/train/images/001_split_0_2_jpg.rf.4ea2cf69d433dc14dd69e0b960debc7b.jpg: 384x640 3 Rice-Panicles, 40.3ms
image 2/1754 /kaggle/working/rice_teacher_student/0_yolo_split/train/images/001_split_1_1_jpg.rf.5683a899aee9046409483ef58d76f44e.jpg: 384x640 (no detections), 7.9ms
image 3/1754 /kaggle/working/rice_teacher_student/0_yolo_split/train/images/001_split_2_1_jpg.r

train: Scanning /kaggle/working/rice_teacher_student/2_student/train/labels... 1754 images, 0 backgrounds, 1540 corrupt: 100%|██████████| 1754/1754 [00:00<00:00, 1952.75it/s]

train: /kaggle/working/rice_teacher_student/2_student/train/images/001_split_0_2_jpg.rf.4ea2cf69d433dc14dd69e0b960debc7b.jpg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (21,) + inhomogeneous part.
train: /kaggle/working/rice_teacher_student/2_student/train/images/001_split_2_1_jpg.rf.8d4a453555d441316606e5f410fc0319.jpg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (16,) + inhomogeneous part.
train: /kaggle/working/rice_teacher_student/2_student/train/images/001_split_2_3_jpg.rf.fa2203dc48133bf3adc6f5605e7470aa.jpg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (21,) + inhomogeneous part.
train: /kaggle/working/rice_teacher_student/2_studen

train: New cache created: /kaggle/working/rice_teacher_student/2_student/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 629.0±470.2 MB/s, size: 147.5 KB)


val: Scanning /kaggle/working/rice_teacher_student/2_student/valid/labels... 219 images, 0 backgrounds, 0 corrupt: 100%|██████████| 219/219 [00:00<00:00, 1819.72it/s]

...

                   all        219       3962      0.615      0.671      0.681      0.405
          Rice-Panicle        219       3962      0.615      0.671      0.681      0.405
Speed: 0.4ms preprocess, 5.1ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to /kaggle/working/rice_teacher_student/student_it3

=== STAC iteration 5/5 ===

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1754 /kaggle/working/rice_teacher_student/0_yolo_split/train/images/001_split_0_2_jpg.rf.4ea2c

      10/10      2.63G      1.333      1.083      1.107         46        640: 100%|██████████| 72/72 [00:12<00:00,  5.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:01<00:00,  8.56it/s]


                   all        219       3962      0.662      0.694      0.723       0.45

10 epochs completed in 0.042 hours.
Optimizer stripped from /kaggle/working/rice_teacher_student/student_it4/weights/last.pt, 22.5MB
Optimizer stripped from /kaggle/working/rice_teacher_student/student_it4/weights/best.pt, 22.5MB

Validating /kaggle/working/rice_teacher_student/student_it4/weights/best.pt...
Ultralytics 8.3.162 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:04<00:00,  2.84it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        219       3962      0.661      0.695      0.723       0.45
          Rice-Panicle        219       3962      0.661      0.695      0.723       0.45
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 10.6ms postprocess per image
Results saved to /kaggle/working/rice_teacher_student/student_it4


In [11]:
# ================================================================
# 7. Final evaluation (no matplotlib warnings, no plots)
# ================================================================
warnings.filterwarnings("ignore", category=RuntimeWarning)
final = YOLO(str(teacher_weights))
metrics = final.val(
    data    = str(data_yaml_path),
    imgsz   = 640,
    batch   = 4,
    device  = 0,
    plots   = False
)
print("\n🏁 STAC‐style final student metrics:", metrics)

Ultralytics 8.3.162 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2498.4±883.0 MB/s, size: 145.9 KB)


val: Scanning /kaggle/working/rice_teacher_student/0_yolo_split/valid/labels.cache... 219 images, 0 backgrounds, 0 corrupt: 100%|██████████| 219/219 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:01<00:00, 29.05it/s]


                   all        219       3962      0.662      0.695      0.723      0.451
          Rice-Panicle        219       3962      0.662      0.695      0.723      0.451
Speed: 0.2ms preprocess, 3.8ms inference, 0.0ms loss, 1.2ms postprocess per image

🏁 STAC‐style final student metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7da6c03d3110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.0260